<a href="https://colab.research.google.com/github/pvfalcao/A-semi-supervised-learning-approach-with-data-drift-detection-to-deal-with-ransomware/blob/main/10s/Rel_RASCO_no_data_drift_10S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas


In [1]:
!pip install sslearn
!pip install frouros

In [2]:
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sslearn
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sslearn.wrapper import RelRasco
from sslearn.model_selection import artificial_ssl_dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from frouros.callbacks.batch import PermutationTestDistanceBased
from frouros.detectors.data_drift import MMD
from sklearn.model_selection import cross_val_score
from sklearn.manifold import TSNE
from frouros.detectors.data_drift import KSTest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


Recebendo conjunto de dados


In [3]:
df = pd.read_csv(r'/content/drive/MyDrive/Navarra/N10S10/N10S10.csv', header = None)
zero = pd.read_csv(r'/content/drive/MyDrive/Navarra/N10S10/zeroDays.csv', header = None)
user = pd.read_csv(r'/content/drive/MyDrive/Navarra/N10S10/userSamples_train.csv', header = None)

targets = df.iloc[:,df.shape[1] - 1]
df = df.iloc[:,0:-1]
tz = zero.iloc[:,zero.shape[1] - 1]
zero = zero.iloc[:,0:-1]
tu = user.iloc[:,user.shape[1] - 1]
user = user.iloc[:,0:-1]


Modelo

In [4]:

#Initialize the variables

precision_list = []
recall_list = []
f1_list = []
accuracy_list = []

# Initialize 10-Fold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = RelRasco()

#Consumption metrics

for lr in [0.1, 0.2, 0.3, 0.4]:

  psutil.cpu_percent(interval=None)
  start = time.process_time()

  # Split the dataset into 10 folds

  for fold, (train_idx, test_idx) in enumerate(skf.split(df, targets)):

      # Select data using .iloc to get pandas DataFrames/Series
      train_features = df.iloc[train_idx]
      train_labels = targets.iloc[train_idx]
      test_features = df.iloc[test_idx]
      test_labels = targets.iloc[test_idx]
      TP = 0
      FP = 0
      TN = 0
      FN = 0
      precision_list = []
      recall_list = []
      f1_list = []
      accuracy_list = []

      X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
      y_train, y_test = targets.iloc[train_idx], targets.iloc[test_idx]

      X_ss, y_ss, X_unl, y_unl = artificial_ssl_dataset(X_train, y_train, lr)
      clf.fit(X_ss, y_ss)

        # Unlabeled score
      print(f"[LR={lr}] Unlabeled score: {clf.score(X_unl, y_unl):.4f}")

      y_pred = clf.predict(X_test)

      TN, FP, FN, TP = confusion_matrix(y_pred, y_test).ravel()

      precision = TP / (TP + FP) if TP + FP > 0 else 0
      recall = TP / (TP + FN) if TP + FN > 0 else 0
      f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
      accuracy = (TN + TP)/(TP + FP + FN + TN)


      accuracy_list.append(accuracy)
      precision_list.append(precision)
      recall_list.append(recall)
      f1_list.append(f1)



  #Consumption measures
  end = time.process_time()
  print('The processing time was:', end - start)
  cpu_usage_system = psutil.cpu_percent(interval=None)
  print(f"System-wide CPU usage: {cpu_usage_system}%")
  print(f"Load average:{psutil.getloadavg()}")
  print(f"CPU Frequency:{psutil.cpu_freq()}")
  print("CPU usage (%):", psutil.cpu_percent())
  print("CPU usage (%):", psutil.cpu_percent())
  ram = psutil.virtual_memory()
  print("RAM usage (%):", ram.percent)
  print("RAM used (GB):", round(ram.used / 1e9, 2))
  mean_precision = sum(precision_list) / len(precision_list)
  mean_recall = sum(recall_list) / len(recall_list)
  mean_f1 = sum(f1_list) / len(f1_list)
  mean_accuracy = sum(accuracy_list) / len(accuracy_list)
  print(f'Mean precision: {mean_precision:.5f}')
  print(f'Mean recall: {mean_recall:.5f}')
  print(f'Mean F1 Score: {mean_f1:.5f}')
  print(f'Mean accuracy: {mean_accuracy:.5f}')







[LR=0.1] Unlabeled score: 0.9963
[LR=0.1] Unlabeled score: 0.9966
[LR=0.1] Unlabeled score: 0.9965
[LR=0.1] Unlabeled score: 0.9962
[LR=0.1] Unlabeled score: 0.9963
[LR=0.1] Unlabeled score: 0.9965
[LR=0.1] Unlabeled score: 0.9965
[LR=0.1] Unlabeled score: 0.9965
[LR=0.1] Unlabeled score: 0.9948
[LR=0.1] Unlabeled score: 0.9945
The processing time was: 123.08410775500002
System-wide CPU usage: 74.3%
Load average:(1.85107421875, 1.48876953125, 0.82080078125)
CPU Frequency:scpufreq(current=2199.998, min=0.0, max=0.0)
CPU usage (%): 0.0
CPU usage (%): 0.0
RAM usage (%): 13.7
RAM used (GB): 1.53
Mean precision: 0.99158
Mean recall: 0.99158
Mean F1 Score: 0.99158
Mean accuracy: 0.99384
[LR=0.2] Unlabeled score: 0.9975
[LR=0.2] Unlabeled score: 0.9977
[LR=0.2] Unlabeled score: 0.9972
[LR=0.2] Unlabeled score: 0.9966
[LR=0.2] Unlabeled score: 0.9975
[LR=0.2] Unlabeled score: 0.9976
[LR=0.2] Unlabeled score: 0.9976
[LR=0.2] Unlabeled score: 0.9977
[LR=0.2] Unlabeled score: 0.9969
[LR=0.2] Unla

Zero day dataset

In [6]:
for lr in [0.1, 0.2, 0.3, 0.4]:
   # Reset TP, FP, FN, TN for each LR value
   TP = 0
   FP = 0
   FN = 0
   TN = 0

   X_ss, y_ss, X_unl, y_unl = artificial_ssl_dataset(X_train, y_train, lr)
   clf.fit(X_ss, y_ss)

   # Unlabeled score
   print(f"[LR={lr}] Unlabeled score: {clf.score(X_unl, y_unl):.4f}")
   y_pred = clf.predict(zero)
   for i in range(tz.shape[0]):
       if y_pred[i] == 0 and tz[i] == 0:
          TN += 1
       elif y_pred[i] == 0 and tz[i] == 1:
          FN += 1
       elif y_pred[i] == 1 and tz[i] == 0:
          FP += 1
       elif y_pred[i] == 1 and tz[i] == 1:
          TP += 1
   precision = TP / (TP + FP) if TP + FP > 0 else 0
   recall = TP / (TP + FN) if TP + FN > 0 else 0
   f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
   accuracy = (TN + TP)/(TP + FP + FN + TN)
   print(f'The results for {lr} are:')
   print(f'Precision: {precision:.5f}')
   print(f'Recall: {recall:.5f}')
   print(f'F1: {f1:.5f}')
   print(f'Accuracy: {accuracy:.5f}')



[LR=0.1] Unlabeled score: 0.9965
The results for 0.1 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000
[LR=0.2] Unlabeled score: 0.9972
The results for 0.2 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000
[LR=0.3] Unlabeled score: 0.9974
The results for 0.3 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000
[LR=0.4] Unlabeled score: 0.9987
The results for 0.4 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000


User sample dataset

In [8]:
for lr in [0.1, 0.2, 0.3, 0.4]:
   # Reset TP, FP, FN, TN for each LR value
   TP = 0
   FP = 0
   FN = 0
   TN = 0

   X_ss, y_ss, X_unl, y_unl = artificial_ssl_dataset(X_train, y_train, lr)
   clf.fit(X_ss, y_ss)

   # Unlabeled score
   print(f"[LR={lr}] Unlabeled score: {clf.score(X_unl, y_unl):.4f}")
   y_pred = clf.predict(user)
   for i in range(tu.shape[0]):
       if y_pred[i] == 0 and tu[i] == 0:
          TN += 1
       elif y_pred[i] == 0 and tu[i] == 1:
          FN += 1
       elif y_pred[i] == 1 and tu[i] == 0:
          FP += 1
       elif y_pred[i] == 1 and tu[i] == 1:
          TP += 1
   precision = TN / (TN + FN) if (TN + FN) > 0 else 0
   recall = TN / (TN + FP) if (TN + FP) > 0 else 0
   f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
   accuracy = (TN + TP)/(TP + FP + FN + TN)
   print(f'The results for {lr} are:')
   print(f'Precision: {precision:.5f}')
   print(f'Recall: {recall:.5f}')
   print(f'F1: {f1:.5f}')
   print(f'Accuracy: {accuracy:.5f}')



[LR=0.1] Unlabeled score: 0.9967
The results for 0.1 are:
Precision: 0.00000
Recall: 0.00000
F1: 0.00000
Accuracy: 0.00000
[LR=0.2] Unlabeled score: 0.9974
The results for 0.2 are:
Precision: 0.00000
Recall: 0.00000
F1: 0.00000
Accuracy: 0.00000
[LR=0.3] Unlabeled score: 0.9977
The results for 0.3 are:
Precision: 0.00000
Recall: 0.00000
F1: 0.00000
Accuracy: 0.00000
[LR=0.4] Unlabeled score: 0.9975
The results for 0.4 are:
Precision: 0.00000
Recall: 0.00000
F1: 0.00000
Accuracy: 0.00000
